In [2]:
import math
import random
import cv2

def get_weight(number_of_neurons_per_layer, number_of_neurons_on_next_per_layer, random_from, random_to):
    # генерируем веса для нейронов
    hidden_neurons_weight = []
    for j in range(number_of_neurons_per_layer):
        weight = []
        for i in range(number_of_neurons_on_next_per_layer):
            weight.append(random.uniform(random_from, random_to))
        hidden_neurons_weight.append(weight)
    print('Начльные веса на втром слое скрытых нейронов')
    for i in hidden_neurons_weight:
        print(i)
    print()
    return hidden_neurons_weight


def sigmoid(input_value):
    return 1 / (1 + math.e ** (-input_value))


def grad_sigmoid(input_value):
    return input_value * (1 - input_value)


def leaky_relu(x):
    if x < 0:
        return 0.1
    elif x >= 0:
        return x


def grad_leaky_relu(x):
    if x < 0:
        return 0.01
    elif x >= 0:
        return 1


def find_answers(answers):
    maximum = -1
    index = -1
    for i, ii in enumerate(answers):
        if ii > maximum:
            maximum = ii
            index = i
    return index


class Neural:
    def __init__(self, data, y, input_neurons_weight, first_hidden_neurons_weight, second_hidden_neurons_weight,
                 learning_step, iteration):
        self.data = data
        self.y = y
        self.input_neurons_weight = input_neurons_weight
        self.first_hidden_neurons_weight = first_hidden_neurons_weight
        self.second_hidden_neurons_weight = second_hidden_neurons_weight
        self.learning_step = learning_step
        self.iteration = iteration

    def train(self):
        epoch = 0
        while epoch != self.iteration + 1:
            self.direct_distribution()
            if epoch % 1000 == 0:
                print()
                print(f'Эпоха - {epoch}')
                print(f'Входной слой веса {self.input_neurons_weight}')
                print(f'Второй скрытый слой веса {self.second_hidden_neurons_weight}')
                print()
            epoch += 1

    def direct_distribution(self):
        values_on_input_first_layer = [0] * len(self.input_neurons_weight[0])
        values_on_input_second_layer = [0] * len(self.first_hidden_neurons_weight[0])
        values_on_out_layer = [0] * len(self.second_hidden_neurons_weight[0])
        for d, dd in enumerate(self.data):  # проход по данным
            for i, ii in enumerate(self.input_neurons_weight[0]):
                for j, jj in enumerate(self.input_neurons_weight):
                    values_on_input_first_layer[i] += \
                        dd[j] * self.input_neurons_weight[j][i]
            values_on_input_first_layer = [sigmoid(i) for i in values_on_input_first_layer]
            # print(f'Значения после второго слоя - {values_on_input_first_layer}')

            for i, ii in enumerate(self.first_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.first_hidden_neurons_weight):
                    values_on_input_second_layer[i] += \
                        values_on_input_first_layer[j] * self.first_hidden_neurons_weight[j][i]
            values_on_input_second_layer = [sigmoid(i) for i in values_on_input_second_layer]
            # print(f'Значения после второго слоя - {values_on_input_first_layer}')

            for i, ii in enumerate(self.second_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.second_hidden_neurons_weight):
                    values_on_out_layer[i] += values_on_input_second_layer[j] * \
                                              self.second_hidden_neurons_weight[j][i]
            values_on_out_layer = [sigmoid(i) for i in values_on_out_layer]
            # print(f'Значения наыходе - {values_on_out_layer}')
            # print()

            # обратное распрастранение ошибки
            # расчёт ошибки на всех уровнях
            answer_error = [0] * len(values_on_out_layer)
            for i, ii in enumerate(answer_error):
                answer_error[i] = self.y[d][i] - values_on_out_layer[i]
            # print('Ошибки сети', answer_error)

            second_layer_error = [0] * len(values_on_input_second_layer)
            input_layer_error = [0] * len(input_neurons_weight)

            for i, ii in enumerate(self.second_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.second_hidden_neurons_weight):
                    second_layer_error[i] += answer_error[i] * self.second_hidden_neurons_weight[j][i]
            # print('Ошибки на втором скрытом слое', second_layer_error)

            for i, ii in enumerate(self.first_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.first_hidden_neurons_weight):
                    input_layer_error[i] += second_layer_error[i] * self.first_hidden_neurons_weight[j][i]
            # print('Ошибки на втором скрытом слое', second_layer_error)

            # обновляем веса
            for i, ii in enumerate(self.input_neurons_weight[0]):
                for j, jj in enumerate(self.input_neurons_weight):
                    self.input_neurons_weight[j][i] += \
                        input_layer_error[i] * grad_sigmoid(values_on_input_first_layer[i]) * \
                        data[d][i] * self.learning_step
            # print(f'Новые веса для входного слоя {self.input_neurons_weight}')

            for i, ii in enumerate(self.first_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.first_hidden_neurons_weight):
                    self.first_hidden_neurons_weight[j][i] += \
                        second_layer_error[i] * grad_sigmoid(values_on_input_second_layer[i]) * \
                        values_on_input_first_layer[j] * self.learning_step
            # print(f'Новые веса для входного слоя {self.input_neurons_weight}')

            for i, ii in enumerate(self.second_hidden_neurons_weight[0]):
                for j, jj in enumerate(self.second_hidden_neurons_weight):
                    self.second_hidden_neurons_weight[j][i] += \
                        answer_error[i] * grad_sigmoid(values_on_out_layer[i]) * \
                        values_on_input_second_layer[j] * self.learning_step
            # print(f'Новые веса для второго слоя {self.input_neurons_weight}')

    def predict(self, new_data):
        values_on_input_first_layer = [0] * len(self.input_neurons_weight[0])
        values_on_input_second_layer = [0] * len(self.first_hidden_neurons_weight[0])
        values_on_out_layer = [0] * len(self.second_hidden_neurons_weight[0])
        for i, ii in enumerate(self.input_neurons_weight[0]):
            for j, jj in enumerate(self.input_neurons_weight):
                values_on_input_first_layer[i] += \
                    new_data[j] * self.input_neurons_weight[j][i]
        values_on_input_first_layer = [sigmoid(i) for i in values_on_input_first_layer]
        # print(f'Значения после второго слоя - {values_on_input_first_layer}')

        for i, ii in enumerate(self.first_hidden_neurons_weight[0]):
            for j, jj in enumerate(self.first_hidden_neurons_weight):
                values_on_input_second_layer[i] += \
                    values_on_input_first_layer[j] * self.first_hidden_neurons_weight[j][i]
        values_on_input_second_layer = [sigmoid(i) for i in values_on_input_second_layer]
        # print(f'Значения после второго слоя - {values_on_input_first_layer}')

        for i, ii in enumerate(self.second_hidden_neurons_weight[0]):
            for j, jj in enumerate(self.second_hidden_neurons_weight):
                values_on_out_layer[i] += values_on_input_second_layer[j] * \
                                          self.second_hidden_neurons_weight[j][i]
        values_on_out_layer = [sigmoid(i) for i in values_on_out_layer]
        answer_neural = find_answers(values_on_out_layer)
        print(f'На вход подалось {new_data}')
        print(f'Ответ нейросети: {answer_neural}')
        print(f'{values_on_out_layer}')
        print()




In [3]:
if __name__ == '__main__':
    input_neurons_weight = get_weight(90, 24, 0., 0.5)
    first_hidden_neurons_weight = get_weight(24, 12, 0., 0.5)
    second_hidden_neurons_weight = get_weight(12, 5, 0., 0.5)
    data, y = get_data_and_y([1, 2, 3, 4, 5])

    multi_layered_thing = Neural(data, y, input_neurons_weight, first_hidden_neurons_weight,
                                 second_hidden_neurons_weight,
                                 learning_step=0.1, iteration=1000)

    multi_layered_thing.train()

    multi_layered_thing.predict(convert_data(1))
    multi_layered_thing.predict(convert_data(3))
    multi_layered_thing.predict(convert_data(5))

Начльные веса на втром слое скрытых нейронов
[0.4907434878116182, 0.17718604498404272, 0.09153605657915553, 0.32465805257589203, 0.4154149923284605, 0.13670881688045333, 0.027339131470955957, 0.043944331238740364, 0.2968691218327006, 0.27249179161926557, 0.14783374612691708, 0.08769940530657411, 0.22985479389709979, 0.1828032856373759, 0.2557437656470381, 0.31253148248536594, 0.08619450946438045, 0.3845572894626358, 0.24565528636822653, 0.14360055870500277, 0.28525180976772835, 0.04581968497696509, 0.018357698007270384, 0.15113154090957276]
[0.35092701565068407, 0.32865975254291135, 0.18613422760563708, 0.2440834340882982, 0.3971884997469972, 0.2812816122408545, 0.2805591839672625, 0.3142769028876838, 0.18401671112534235, 0.08194820055776125, 0.01761239501041828, 0.4389540476004749, 0.13938937212061958, 0.20736982013331706, 0.28897052969542597, 0.37473992169863835, 0.36646986452514385, 0.34756661257983124, 0.16761302133147432, 0.48637759263289115, 0.1273401347970723, 0.1464967708904102

Новая версия
Новая версия
Новая версия
Новая версия
Новая версия

In [9]:
if __name__ == '__main__':
    input_neurons_weight = get_weight(15, 10, 0., 0.5)
    first_hidden_neurons_weight = get_weight(10, 5, 0., 0.5)
    second_hidden_neurons_weight = get_weight(5, 10, 0., 0.5)
    data, y = get_x_y()

    multi_layered_thing = Neural(data, y, input_neurons_weight, first_hidden_neurons_weight,
                                 second_hidden_neurons_weight,
                                 learning_step=0.1, iteration=1000)

    multi_layered_thing.train()

    multi_layered_thing.predict(convert_data(1))
    multi_layered_thing.predict(convert_data(3))
    multi_layered_thing.predict(convert_data(5))

Начльные веса на втром слое скрытых нейронов
[0.46074405600392077, 0.28485300756645654, 0.22315199592790946, 0.02175219417855545, 0.17841390739699603, 0.15178597823963058, 0.46428050254332787, 0.31552699942823986, 0.34227729565861476, 0.41864708705758175]
[0.4920365233115209, 0.03545528117904173, 0.25422865952419915, 0.16632455382801492, 0.03744598341127414, 0.3577551729008627, 0.4405134739697316, 0.3079668377871441, 0.25246656482803886, 0.13252613540957547]
[0.41523705873145694, 0.2999182325848927, 0.09452183731080727, 0.4588091952686527, 0.15713828508237238, 0.005419741740914863, 0.4279547333317827, 0.2053223928613418, 0.3014916828964931, 0.3440609667707448]
[0.23271252361171552, 0.35003842239792937, 0.2679508567448854, 0.14659901097380112, 0.1571698973488478, 0.0683825858057302, 0.31289574950438226, 0.3658443671534573, 0.391530493362873, 0.23427313067662442]
[0.3436094284610854, 0.007627798770637961, 0.0771640953912458, 0.22833090723795818, 0.30110619330604116, 0.32309956874226786, 

IndexError: list index out of range

In [10]:
def get_x_y():
    x = [[1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1],
            [0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1],
            [1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1],
            [1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1],
            [1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1],
            [1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1],
            [1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1],
            [1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1]]
    y = [[1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
         [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]
    return x, y